# 04 - Pipeline Final de Produção

**🎯 PROPÓSITO:**  
Este notebook implementa a solução final para gerar as predições de submissão do hackathon.

**🚀 EXECUÇÃO:**  
Execute todas as células em sequência. O arquivo `submission.csv` e `submission.parquet` serão gerados automaticamente.

**📊 MODELO ESCOLHIDO:**  
LightGBM - selecionado após rigorosa comparação no notebook `03-Modeling-Experiments.ipynb`.

---

## Pipeline Completo:
1. **Carregar dados processados** (do notebook 02)
2. **Otimizar memória** com downcasting
3. **Treinar LightGBM** no dataset completo de 2022  
4. **Implementar previsão iterativa** para as 5 semanas de 2023
5. **Gerar submissão** nos formatos CSV e Parquet

In [1]:
# Importações necessárias
import pandas as pd
import numpy as np
import pickle
import warnings
warnings.filterwarnings('ignore')

# ML Libraries
import lightgbm as lgb
import gc

# Time series
from datetime import datetime, timedelta
import os

print('📚 Bibliotecas carregadas com sucesso!')
print('🚀 Iniciando Pipeline Final de Produção')

📚 Bibliotecas carregadas com sucesso!
🚀 Iniciando Pipeline Final de Produção


## 1. Carregamento e Preparação dos Dados

In [2]:
# Verificar arquivos necessários
required_files = [
    '../data/dados_features_completo.parquet',
    '../data/feature_engineering_metadata.pkl'
]

missing_files = [f for f in required_files if not os.path.exists(f)]
if missing_files:
    raise FileNotFoundError(f'Arquivos não encontrados: {missing_files}')

print('📂 Carregando dados processados...')
dados = pd.read_parquet('../data/dados_features_completo.parquet')

with open('../data/feature_engineering_metadata.pkl', 'rb') as f:
    metadata = pickle.load(f)

print(f'✅ Dados carregados: {dados.shape}')
print(f'📅 Período: {dados["semana"].min()} até {dados["semana"].max()}')
print(f'💾 Memória inicial: {dados.memory_usage(deep=True).sum() / (1024**2):.1f} MB')

📂 Carregando dados processados...
✅ Dados carregados: (51171190, 26)
📅 Período: 2022-01-25 00:00:00 até 2022-12-27 00:00:00
💾 Memória inicial: 16045.8 MB


## 2. Otimização de Memória

In [3]:
print('🔧 Aplicando otimização de memória (downcasting)...')

# Downcasting de tipos numéricos
for col in dados.select_dtypes(include=[np.number]).columns:
    if dados[col].dtype.kind in ['i', 'u']:  # Inteiros
        dados[col] = pd.to_numeric(dados[col], downcast='integer')
    else:  # Floats
        dados[col] = pd.to_numeric(dados[col], downcast='float')

# Otimizar categóricas
for col in dados.select_dtypes(include=['object']).columns:
    if col not in ['semana']:
        nunique = dados[col].nunique()
        if nunique / len(dados) < 0.5:
            dados[col] = dados[col].astype('category')

# Tratar missing values
if 'distributor_id' in dados.columns:
    if dados['distributor_id'].dtype.name == 'category':
        if -1 not in dados['distributor_id'].cat.categories:
            dados['distributor_id'] = dados['distributor_id'].cat.add_categories([-1])
    dados['distributor_id'] = dados['distributor_id'].fillna(-1)

# Preencher outros NaNs
for col in dados.columns:
    if dados[col].isnull().sum() > 0 and dados[col].dtype.kind in ['i', 'u', 'f']:
        dados[col] = dados[col].fillna(0)

memory_optimized = dados.memory_usage(deep=True).sum() / (1024**2)
print(f'✅ Memória otimizada: {memory_optimized:.1f} MB')
print(f'📈 Redução de memória: {((memory_optimized / (dados.memory_usage(deep=True).sum() / (1024**2)) - 1) * 100):.1f}%')

🔧 Aplicando otimização de memória (downcasting)...
✅ Memória otimizada: 4491.9 MB
📈 Redução de memória: 0.0%


## 3. Preparação para Treinamento

In [4]:
# Definir features e target
target = 'quantidade'
exclude_features = ['pdv_id', 'produto_id', 'semana', 'quantidade', 'valor', 'num_transacoes']
all_features = [col for col in dados.columns if col not in exclude_features]

print(f'🎯 Configuração do modelo:')
print(f'   • Target: {target}')
print(f'   • Features: {len(all_features)}')

# Preparar dados para treinamento (todo o dataset de 2022)
X_train = dados[all_features]
y_train = dados[target]

print(f'📊 Dados de treinamento:')
print(f'   • Shape: {X_train.shape}')
print(f'   • Período completo de 2022')

🎯 Configuração do modelo:
   • Target: quantidade
   • Features: 20
📊 Dados de treinamento:
   • Shape: (51171190, 20)
   • Período completo de 2022


## 4. Treinamento do Modelo LightGBM

In [ ]:
# PASSO B: LightGBM Vanilla - Validar Pipeline e Features
print('\n🚀 PASSO B: Modelo LightGBM Vanilla (Parâmetros Default)')
print('=' * 60)
print('🎯 Objetivo: Validar se nossas features têm poder preditivo')

# Configuração LightGBM Vanilla (parâmetros simples/default)
lgb_params_vanilla = {
    'objective': 'regression_l1',  # MAE - melhor para WMAPE
    'metric': 'mae',
    'boosting_type': 'gbdt',
    'verbosity': -1,
    'random_state': 42,
    'n_jobs': -1
}

print('\n📋 Configuração Vanilla:')
for param, value in lgb_params_vanilla.items():
    print(f'   • {param}: {value}')

# Preparar dados para LightGBM
print('\n📊 Preparando dados para treinamento...')
train_lgb = lgb.Dataset(X_train, label=y_train)

print(f'   • Train shape: {X_train.shape}')
print(f'   • Features: {len(all_features)}')

# Treinar modelo Vanilla (VERSÃO PARA PRODUÇÃO - SEM VALIDAÇÃO)
print('\n🔄 Treinando LightGBM Vanilla (produção - dataset completo)...')
lgb_vanilla = lgb.train(
    lgb_params_vanilla,
    train_lgb,
    num_boost_round=200  # Número moderado para vanilla
)

print(f'✅ Treinamento concluído em {lgb_vanilla.num_trees()} iterações')

# Salvar modelo
os.makedirs('../models', exist_ok=True)
lgb_vanilla.save_model('../models/lightgbm_final.txt')
print('💾 Modelo salvo em: ../models/lightgbm_final.txt')

print('\n✅ LightGBM treinado e salvo com sucesso!')


🚀 PASSO B: Modelo LightGBM Vanilla (Parâmetros Default)
🎯 Objetivo: Validar se nossas features têm poder preditivo

📋 Configuração Vanilla:
   • objective: regression_l1
   • metric: mae
   • boosting_type: gbdt
   • verbosity: -1
   • random_state: 42
   • n_jobs: -1

📊 Preparando dados para treinamento...
   • Train shape: (51171190, 20)
   • Features: 20

🔄 Treinando LightGBM Vanilla (produção - dataset completo)...


## 5. Implementação da Previsão Iterativa

In [ ]:
print('🔄 Implementando previsão iterativa para 2023...')

# Obter combinações únicas de PDV x Produto de 2022
combinacoes_2022 = dados[['pdv_id', 'produto_id']].drop_duplicates()
print(f'📊 Combinações PDV x Produto encontradas em 2022: {len(combinacoes_2022):,}')

# Definir as 5 semanas de 2023 para previsão
# Assumindo que 2023 começa na semana seguinte ao último dado de 2022
ultima_semana_2022 = dados['semana'].max()
semanas_2023 = []
for i in range(1, 6):  # 5 semanas
    proxima_semana = ultima_semana_2022 + timedelta(weeks=i)
    semanas_2023.append(proxima_semana)

print('📅 Semanas para previsão:')
for i, semana in enumerate(semanas_2023, 1):
    print(f'   {i}. {semana.strftime("%Y-%m-%d")}')

# Preparar dados históricos para features iterativas
dados_historicos = dados.copy()
previsoes_finais = []

print('\\n🔄 Iniciando previsões iterativas...')
for week_num, semana_target in enumerate(semanas_2023, 1):
    print(f'\\n   Semana {week_num}/5: {semana_target.strftime("%Y-%m-%d")}')
    
    # Criar features para a semana alvo
    dados_semana = combinacoes_2022.copy()
    dados_semana['semana'] = semana_target
    
    # Engenharia de features baseada no histórico atual
    print('     🔧 Criando features...')
    
    # Features temporais
    dados_semana['mes_sin'] = np.sin(2 * np.pi * semana_target.month / 12)
    dados_semana['mes_cos'] = np.cos(2 * np.pi * semana_target.month / 12)
    
    # Features de lag (baseadas no histórico)
    for lag in [1, 2, 3, 4]:
        semana_lag = semana_target - timedelta(weeks=lag)
        dados_lag = dados_historicos[dados_historicos['semana'] == semana_lag]
        
        if len(dados_lag) > 0:
            lag_dict = dados_lag.set_index(['pdv_id', 'produto_id'])['quantidade'].to_dict()
            dados_semana[f'quantidade_lag_{lag}'] = dados_semana.apply(
                lambda row: lag_dict.get((row['pdv_id'], row['produto_id']), 0), axis=1
            )
        else:
            dados_semana[f'quantidade_lag_{lag}'] = 0
    
    # Features de rolling (últimas 4 semanas)
    semanas_rolling = [semana_target - timedelta(weeks=i) for i in range(1, 5)]
    dados_rolling = dados_historicos[dados_historicos['semana'].isin(semanas_rolling)]
    
    if len(dados_rolling) > 0:
        rolling_stats = dados_rolling.groupby(['pdv_id', 'produto_id'])['quantidade'].agg({
            'mean': 'mean',
            'max': 'max', 
            'min': 'min'
        }).reset_index()
        
        rolling_stats.columns = ['pdv_id', 'produto_id', 'quantidade_media_4w', 'quantidade_max_4w', 'quantidade_min_4w']
        dados_semana = dados_semana.merge(rolling_stats, on=['pdv_id', 'produto_id'], how='left')
    
    # Preencher NaNs
    for col in ['quantidade_media_4w', 'quantidade_max_4w', 'quantidade_min_4w']:
        if col in dados_semana.columns:
            dados_semana[col] = dados_semana[col].fillna(0)
    
    # Features de hash e históricas (copiar lógica do notebook de feature engineering)
    dados_semana['pdv_hash'] = dados_semana['pdv_id'].astype(str).apply(hash) % 100
    dados_semana['produto_hash'] = dados_semana['produto_id'].astype(str).apply(hash) % 100
    dados_semana['pdv_produto_hash'] = (dados_semana['pdv_id'].astype(str) + '_' + dados_semana['produto_id'].astype(str)).apply(hash) % 1000
    
    # Features históricas (toda a série até agora)
    hist_stats = dados_historicos.groupby(['pdv_id', 'produto_id'])['quantidade'].agg({
        'mean': 'mean',
        'std': 'std',
        'max': 'max',
        'count': 'count'
    }).reset_index()
    hist_stats.columns = ['pdv_id', 'produto_id', 'hist_mean', 'hist_std', 'hist_max', 'hist_count']
    dados_semana = dados_semana.merge(hist_stats, on=['pdv_id', 'produto_id'], how='left')
    
    # Preencher NaNs
    for col in ['hist_mean', 'hist_std', 'hist_max', 'hist_count']:
        dados_semana[col] = dados_semana[col].fillna(0)
    
    # Adicionar distributor_id (assumir como -1 para novas predições)
    dados_semana['distributor_id'] = -1
    
    # Garantir que todas as features necessárias existem
    for feature in all_features:
        if feature not in dados_semana.columns:
            dados_semana[feature] = 0
    
    # Fazer previsões
    print('     🎯 Fazendo previsões...')
    X_pred = dados_semana[all_features]
    
    # Garantir tipos de dados consistentes
    for col in X_pred.columns:
        if X_pred[col].dtype != X_train[col].dtype:
            X_pred[col] = X_pred[col].astype(X_train[col].dtype)
    
    # CORREÇÃO: Usar lgb_vanilla em vez de model
    predictions = lgb_vanilla.predict(X_pred)
    predictions = np.maximum(0, predictions)  # Não permitir valores negativos
    
    # Armazenar previsões
    resultado_semana = dados_semana[['pdv_id', 'produto_id', 'semana']].copy()
    resultado_semana['quantidade'] = predictions
    previsoes_finais.append(resultado_semana)
    
    # Atualizar dados históricos com as previsões
    dados_historicos = pd.concat([dados_historicos, resultado_semana], ignore_index=True)
    
    print(f'     ✅ {len(predictions):,} previsões geradas')

print('\\n✅ Previsão iterativa concluída!')

## 6. Geração dos Arquivos de Submissão

In [ ]:
print('📋 Preparando arquivos de submissão...')

# Consolidar todas as previsões
submission_df = pd.concat(previsoes_finais, ignore_index=True)

print(f'📊 Estatísticas das previsões:')
print(f'   • Total de registros: {len(submission_df):,}')
print(f'   • Combinações únicas: {submission_df.groupby(["pdv_id", "produto_id"]).size().shape[0]:,}')
print(f'   • Semanas: {submission_df["semana"].nunique()}')
print(f'   • Quantidade média: {submission_df["quantidade"].mean():.4f}')
print(f'   • Quantidade máxima: {submission_df["quantidade"].max():.4f}')
print(f'   • % zeros: {(submission_df["quantidade"] == 0).mean()*100:.1f}%')

# Criar diretório de submissão
os.makedirs('../submission', exist_ok=True)

# Salvar em CSV
csv_path = '../submission/submission.csv'
submission_df.to_csv(csv_path, index=False)
print(f'💾 Arquivo CSV salvo: {csv_path}')

# Salvar em Parquet
parquet_path = '../submission/submission.parquet'
submission_df.to_parquet(parquet_path, index=False)
print(f'💾 Arquivo Parquet salvo: {parquet_path}')

# Verificar tamanhos dos arquivos
csv_size = os.path.getsize(csv_path) / (1024**2)
parquet_size = os.path.getsize(parquet_path) / (1024**2)

print(f'\n📏 Tamanhos dos arquivos:')
print(f'   • CSV: {csv_size:.1f} MB')
print(f'   • Parquet: {parquet_size:.1f} MB ({parquet_size/csv_size*100:.1f}% do CSV)')

## 7. Validação e Resumo Final

In [ ]:
print('🔍 Validação final dos dados...')

# Verificar estrutura dos dados
print(f'\n📋 Estrutura final:')
print(f'   • Colunas: {list(submission_df.columns)}')
print(f'   • Tipos: {submission_df.dtypes.to_dict()}')
print(f'   • Shape: {submission_df.shape}')

# Verificar completude
print(f'\n✅ Verificações:')
print(f'   • Valores nulos: {submission_df.isnull().sum().sum()} (deve ser 0)')
print(f'   • Valores negativos: {(submission_df["quantidade"] < 0).sum()} (deve ser 0)')
print(f'   • Semanas únicas: {sorted(submission_df["semana"].dt.strftime("%Y-%m-%d").unique())}')

# Análise por semana
print(f'\n📊 Análise por semana:')
weekly_stats = submission_df.groupby('semana')['quantidade'].agg({
    'count': 'count',
    'mean': 'mean',
    'sum': 'sum',
    'zeros': lambda x: (x == 0).sum()
}).round(4)

for semana, stats in weekly_stats.iterrows():
    zeros_pct = (stats['zeros'] / stats['count']) * 100
    print(f'   • {semana.strftime("%Y-%m-%d")}: {stats["count"]:,} registros, média={stats["mean"]:.4f}, zeros={zeros_pct:.1f}%')

print(f'\n🎉 PIPELINE FINAL CONCLUÍDO COM SUCESSO!')
print('=' * 60)
print(f'✅ Modelo LightGBM treinado com {len(dados):,} registros de 2022')
print(f'✅ Previsões iterativas geradas para 5 semanas de 2023')
print(f'✅ {len(submission_df):,} previsões salvas em CSV e Parquet')
print(f'✅ Arquivos prontos para submissão no hackathon!')

print(f'\n📁 Arquivos gerados:')
print(f'   • {csv_path}')
print(f'   • {parquet_path}')
print(f'   • ../models/lightgbm_final.txt')